Downloading the Dataset

In [ ]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!rm -r aclImdb/train/unsup

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  31.6M      0  0:00:02  0:00:02 --:--:-- 31.6M


**Preparing the data**

In [ ]:
import os, pathlib, shutil, random
from tensorflow import keras
batch_size = 32
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_n = base_dir / "train_n"
train_dir = base_dir / "train"
for category in ("neg", "pos"):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = 10000
    val_files = files[-num_val_samples:]
    for fname in val_files:
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)

In [ ]:
shutil.rmtree('aclImdb/val')

In [ ]:
shutil.rmtree('aclImdb/train_n')

In [ ]:
for category in ("neg", "pos"):
    os.makedirs(train_n / category)
    files = os.listdir(train_dir / category)
    num_train_samples=250
    train_files = files[-num_train_samples:]
    for fname in train_files:
        shutil.move(train_dir / category / fname,
                    train_n / category / fname)

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train_n", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)
text_only_train_ds = train_ds.map(lambda x, y: x)

Found 500 files belonging to 2 classes.
Found 20000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


**Preparing integer sequence datasets**

In [ ]:
from tensorflow.keras import layers

max_length = 150
max_tokens = 10000
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

**A sequence model built on one-hot encoded vector sequences**

In [ ]:
import tensorflow as tf
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = tf.one_hot(inputs, depth=max_tokens)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_17 (InputLayer)       [(None, None)]            0         
                                                                 
 tf.one_hot_4 (TFOpLambda)   (None, None, 10000)       0         
                                                                 
 bidirectional_16 (Bidirecti  (None, 64)               2568448   
 onal)                                                           
                                                                 
 dropout_16 (Dropout)        (None, 64)                0         
                                                                 
 dense_16 (Dense)            (None, 1)                 65        
                                                                 
Total params: 2,568,513
Trainable params: 2,568,513
Non-trainable params: 0
________________________________________________

**Training a first basic sequence model**

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint("one_hot_bidir_lstm.keras",
                                    save_best_only=True,
                                    monitor="val_loss")
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("one_hot_bidir_lstm.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
16/16 [==============================] - 12s 491ms/step - loss: 0.8876 - accuracy: 0.4900 - val_loss: 0.7603 - val_accuracy: 0.5196
Epoch 2/10
16/16 [==============================] - 9s 571ms/step - loss: 0.6480 - accuracy: 0.6080 - val_loss: 0.7000 - val_accuracy: 0.5461
Epoch 3/10
16/16 [==============================] - 13s 855ms/step - loss: 0.5273 - accuracy: 0.7720 - val_loss: 0.6915 - val_accuracy: 0.5756
Epoch 4/10
16/16 [==============================] - 8s 509ms/step - loss: 0.3671 - accuracy: 0.8940 - val_loss: 0.7142 - val_accuracy: 0.6078
Epoch 5/10
16/16 [==============================] - 7s 427ms/step - loss: 0.2176 - accuracy: 0.9440 - val_loss: 0.7665 - val_accuracy: 0.6277
Epoch 6/10
16/16 [==============================] - 13s 837ms/step - loss: 0.1007 - accuracy: 0.9860 - val_loss: 0.8355 - val_accuracy: 0.6406
Epoch 7/10
16/16 [==============================] - 7s 458ms/step - loss: 0.0590 - accuracy: 0.9940 - val_loss: 1.0058 - val_accuracy: 0.6411
Epo

**Instantiating an `Embedding` layer**

In [ ]:
embedding_layer = layers.Embedding(input_dim=max_tokens, output_dim=256)

**Model that uses an `Embedding` layer trained from scratch**

In [ ]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()



Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, None)]            0         
                                                                 
 embedding_9 (Embedding)     (None, None, 256)         2560000   
                                                                 
 bidirectional_13 (Bidirecti  (None, 64)               73984     
 onal)                                                           
                                                                 
 dropout_13 (Dropout)        (None, 64)                0         
                                                                 
 dense_13 (Dense)            (None, 1)                 65        
                                                                 
Total params: 2,634,049
Trainable params: 2,634,049
Non-trainable params: 0
________________________________________________

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru.keras",
                                    save_best_only=True,
                                    monitor="val_loss")
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_gru.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
16/16 [==============================] - 11s 519ms/step - loss: 0.3594 - accuracy: 0.9060 - val_loss: 0.7083 - val_accuracy: 0.6090
Epoch 2/10
16/16 [==============================] - 7s 466ms/step - loss: 0.2284 - accuracy: 0.9400 - val_loss: 0.7241 - val_accuracy: 0.6263
Epoch 3/10
16/16 [==============================] - 8s 526ms/step - loss: 0.1211 - accuracy: 0.9860 - val_loss: 0.9134 - val_accuracy: 0.6340
Epoch 4/10
16/16 [==============================] - 7s 459ms/step - loss: 0.0653 - accuracy: 0.9880 - val_loss: 0.7718 - val_accuracy: 0.6361
Epoch 5/10
16/16 [==============================] - 8s 513ms/step - loss: 0.0596 - accuracy: 1.0000 - val_loss: 0.8978 - val_accuracy: 0.6515
Epoch 6/10
16/16 [==============================] - 13s 822ms/step - loss: 0.0319 - accuracy: 0.9980 - val_loss: 1.0315 - val_accuracy: 0.6561
Epoch 7/10
16/16 [==============================] - 12s 797ms/step - loss: 0.0234 - accuracy: 1.0000 - val_loss: 1.0428 - val_accuracy: 0.6564
Epo

**Using an `Embedding` layer with masking enabled**

In [ ]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(
    input_dim=max_tokens, output_dim=256, mask_zero=True)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()



Model: "model_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, None)]            0         
                                                                 
 embedding_10 (Embedding)    (None, None, 256)         2560000   
                                                                 
 bidirectional_14 (Bidirecti  (None, 64)               73984     
 onal)                                                           
                                                                 
 dropout_14 (Dropout)        (None, 64)                0         
                                                                 
 dense_14 (Dense)            (None, 1)                 65        
                                                                 
Total params: 2,634,049
Trainable params: 2,634,049
Non-trainable params: 0
________________________________________________

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru_with_masking.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_gru_with_masking.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
16/16 [==============================] - 11s 557ms/step - loss: 0.2279 - accuracy: 0.9380 - val_loss: 0.7286 - val_accuracy: 0.6213
Epoch 2/10
16/16 [==============================] - 8s 488ms/step - loss: 0.1259 - accuracy: 0.9840 - val_loss: 0.9258 - val_accuracy: 0.6284
Epoch 3/10
16/16 [==============================] - 9s 570ms/step - loss: 0.0789 - accuracy: 0.9900 - val_loss: 0.8392 - val_accuracy: 0.6460
Epoch 4/10
16/16 [==============================] - 13s 849ms/step - loss: 0.0511 - accuracy: 0.9940 - val_loss: 0.8997 - val_accuracy: 0.6504
Epoch 5/10
16/16 [==============================] - 8s 518ms/step - loss: 0.0398 - accuracy: 0.9980 - val_loss: 0.8660 - val_accuracy: 0.6489
Epoch 6/10
16/16 [==============================] - 7s 437ms/step - loss: 0.0247 - accuracy: 1.0000 - val_loss: 1.1491 - val_accuracy: 0.6492
Epoch 7/10
16/16 [==============================] - 8s 490ms/step - loss: 0.0169 - accuracy: 1.0000 - val_loss: 1.2431 - val_accuracy: 0.6464
Epoc

#### Using pretrained word embeddings

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2023-04-16 17:06:02--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-04-16 17:06:02--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-04-16 17:06:02--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

**Parsing the GloVe word-embeddings file and Preparing the GloVe word-embeddings matrix**

In [ ]:
import numpy as np
path_to_glove_file = "glove.6B.100d.txt"

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print(f"Found {len(embeddings_index)} word vectors.")

Found 400000 word vectors.


In [ ]:
embedding_dim = 100

vocabulary = text_vectorization.get_vocabulary()
word_index = dict(zip(vocabulary, range(len(vocabulary))))

embedding_matrix = np.zeros((max_tokens, embedding_dim))
for word, i in word_index.items():
    if i < max_tokens:
        embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
embedding_layer = layers.Embedding(
    max_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
    mask_zero=True,
)

**Model that uses a pretrained Embedding layer**

In [ ]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = embedding_layer(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("glove_embeddings_sequence_model.keras",
                                    save_best_only=True,
                                    monitor="val_loss")
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("glove_embeddings_sequence_model.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_21 (InputLayer)       [(None, None)]            0         
                                                                 
 embedding_8 (Embedding)     (None, None, 256)         2560000   
                                                                 
 bidirectional_20 (Bidirecti  (None, 64)               73984     
 onal)                                                           
                                                                 
 dropout_20 (Dropout)        (None, 64)                0         
                                                                 
 dense_20 (Dense)            (None, 1)                 65        
                                                                 
Total params: 2,634,049
Trainable params: 2,634,049
Non-trainable params: 0
________________________________________________